# About:

Prepare the data to populate the 'TexteLégalExactCode' column from the 'Textes' table.

In French-speaking countries :
Legislation includes Acts (Lois) and Ordinances (Ordonnances) :
- A Loi is a text written and passed by parliament.
- An Ordonnance is a text written by the government in the area of the law which must be ratified by parliament.
- In some countries (mainly the DRC), there may have been Ordonnance-Loi or Décret-Loi. I think it's an old name for Ordonnances.
Regulations include Decrees (Décrets) and Orders (Arrêtés) :
- A Décret is a text issued by the government in the Council of Ministers.
- An Arrêté ministériel is a text issued by one ministers. An Arrêté interministériel is a text issued by several ministers.
- A Circulaire (Circular) ou Note de service (Memo) is a text issued by the administration.

In English-speaking countries : (I'm less familiar with it)
Legislation includes Acts and Ordinances :
- An Act is a text written and passed by parliament.
- In some countries, there may have been Ordinances. It seems that sometimes these are local government texts.
- In some countries, there may have been Laws. I think that's the old name for the Acts.
Regulations include Legal Instruments, Legal Notice and Orders : (The exact names depend on the country)
- A Statutory Instrument, Legislation Instrument, Legal Instrument, Government Note or Legal Notice is a text issued by the government.
- An Order seems to come from a minister.

However, we need above all to adapt to the practices of each country. The easiest way is probably to first try to extract the full name from the text : Ordonnance n°2019-022/P-RM du 27 septembre 2019 portant code minier en République du Mali ; Minerals and Mining Act, 2006 (Act 703 of Ghana), etc. It is in the full name of the text that we find the exact nature of the text.


In [5]:
french_definitions = """
Loi: A law passed by the legislature (parliament) in a civil law system. It refers to a formal statute enacted by the legislative body.
Ordonnance: A type of executive order or regulation, often issued by a government authority, and sometimes used to expedite legislative procedures.
Ordonnance-Loi: A law that is issued by executive decree, usually in extraordinary situations, that has the same force as a law passed by the legislature.
Décret: An executive decree issued by the president or a government authority, implementing laws or regulations. It often provides details or guidance on how a law should be applied.
Décret-Loi: A decree issued by the executive that carries the weight of law, typically used in emergency situations or where the legislative process is bypassed.
Arrêté: A legal order issued by an administrative authority (such as a mayor or governor), generally more localized in scope than a decree.
Arrêté ministériel: An order issued by a minister, specifying how certain laws or regulations should be applied within the minister’s domain.
Arrêté interministériel: An order issued jointly by several ministers to regulate a matter that involves multiple ministries.
Circulaire: A circular issued by a government or administrative authority to guide how certain laws or regulations should be interpreted or enforced, usually without binding legal force.
"""

english_definitions = """
Act: A statute or formal written law passed by a legislative body (such as a parliament or congress). It has the highest level of legal authority.
Law: A system of rules created and enforced by governmental or social institutions. It can refer broadly to legal principles, or specifically to individual statutes.
Ordinance: A law or regulation enacted by a municipal or local government. It is more localized than national laws or Acts of Parliament.
Decree: An order issued by a government authority, often in an emergency or under special circumstances, that has the force of law without needing legislative approval.
Statutory Instrument: A form of delegated or secondary legislation made by an individual or body under powers given by an Act of Parliament. It allows the details of an Act to be filled out by regulations.
Legislation Instrument: A general term for legal documents that have the effect of law, including Acts, Statutory Instruments, and other forms of formal legislation.
Legal Instrument: A formal written document that has legal effect. It can include contracts, wills, statutes, decrees, etc.
Government Notice: A formal publication by a government to inform the public about new laws, regulations, or other official actions.
Legal Notice: A formal notification or announcement that has legal implications, often used to inform individuals or the public about legal processes, obligations, or rights.
Order: A formal directive issued by an authority (such as a court or a government official) that has legal force.
"""

In [6]:
%reload_ext autoreload
%autoreload 2

import os, sys

sys.path.append(os.path.abspath(os.pardir))

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [8]:
# COUNTRY_NAME = 'MLI Mali'
COUNTRY_NAME = os.environ.get("COUNTRY_NAME")
SPOKEN_LANGUAGE = "French"

table_column_name = "TexteLegalExactCode"

print(COUNTRY_NAME)

BFA Burkina Faso


In [9]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print('country name OK')

country name OK


In [10]:
text_definitions = french_definitions if "french" in SPOKEN_LANGUAGE.lower() or "portuguese" in SPOKEN_LANGUAGE.lower() else english_definitions

In [11]:
from sql_files import sql_files
from llm_setup import *

# Get legal documents from existing paperless Postgres

In [12]:
df, documents = get_country_legal_docs(COUNTRY_NAME)

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Get the definitions of the Textes Legaux Exactes from the paperless Postgres

In [13]:
df_leg_exacts = get_postgress_data(sql_files['get_textes_legaux_exacts'], db='Ferdi')

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [14]:
df_leg_exacts.head()

,TexteLégalExactCode,TexteLégalStandardCode,TexteLégalExactCodeCourt,TexteLégalExactComplet
0,Lég_Loi_Loi,Lég_Loi,Loi,Loi
1,Lég_Ordonnance_Ordonnance,Lég_Ordonnance,Ordonnance,Ordonnance
2,Lég_Ordonnance_OrdonnanceLoi,Lég_Ordonnance,OrdonnanceLoi,Ordonnance-Loi
3,Rég_Décret_Décret,Rég_Décret,Décret,Décret
4,Rég_Décret_DécretLoi,Rég_Décret,DécretLoi,Décret-Loi


In [15]:
df_leg_exacts['TexteLégalExactComplet'] = df_leg_exacts['TexteLégalExactComplet'].str.lower()

In [16]:
textes_legaux_exacts = [
    'Loi',
    'Act',
    'Law',
    'Ordonnance',
    'Ordonnance-Loi',
    'Ordinance',
    'Décret',
    'Décret-Loi',
    'Decree',
    'Statutory Instrument',
    'Legislation Instrument',
    'Legal Instrument',
    'Government Notice',
    'Legal Notice',
    'Arrêté',
    'Arrêté ministériel',
    'Arrêté interministériel',
    'Order',
    'Circulaire',
]

# Match by regex and title contents

In [17]:
from postgres_connection import get_postgress_data
from sql_files import sql_files
import pandas as pd

In [18]:
print(text_definitions)


Loi: A law passed by the legislature (parliament) in a civil law system. It refers to a formal statute enacted by the legislative body.
Ordonnance: A type of executive order or regulation, often issued by a government authority, and sometimes used to expedite legislative procedures.
Ordonnance-Loi: A law that is issued by executive decree, usually in extraordinary situations, that has the same force as a law passed by the legislature.
Décret: An executive decree issued by the president or a government authority, implementing laws or regulations. It often provides details or guidance on how a law should be applied.
Décret-Loi: A decree issued by the executive that carries the weight of law, typically used in emergency situations or where the legislative process is bypassed.
Arrêté: A legal order issued by an administrative authority (such as a mayor or governor), generally more localized in scope than a decree.
Arrêté ministériel: An order issued by a minister, specifying how certain l

In [19]:
def extract_kw(title):
    result = list()
    categories = ["Loi", "Décret", "Ordonnance", "Circulaire", "Arrêté"]
    for categ in categories:
        if categ in title:
            result.append(categ.lower())
    
    if result:
        return ", ".join(sorted(result))

df['TexteLégalExactCode'] = df.title.apply(lambda x: extract_kw(x))

df['TexteLégalExactCode'] = df['TexteLégalExactCode'].replace("loi, ordonnance", "ordonance-loi")
df['TexteLégalExactCode'] = df['TexteLégalExactCode'].replace("décret, loi", "décret-loi")
df['TexteLégalExactCode'] = df['TexteLégalExactCode'].replace("circulaire, loi", "circulaire")

df['TexteLégalExactCode'].value_counts(dropna=0)


TexteLégalExactCode
loi       71
décret    29
None       2
arrêté     2
Name: count, dtype: int64

In [20]:
df_i = pd.merge(df, df_leg_exacts, how='left', left_on='TexteLégalExactCode', right_on='TexteLégalExactComplet', suffixes=['_', ''])
# df.drop(df_leg_exacts.columns + ['content'], axis=1, inplace=True)
df['TexteLégalExactCode'] = df_i['TexteLégalExactCode']

In [25]:
df[['title', 'country', 'TexteLégalExactCode']].to_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv", index=False)

## Load existing categories

In [26]:
# COUNTRY_NAME = 'MLI Mali'

large_categs = pd.read_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv")
large_categs = list(list(large_categs.set_index(['title'])[['TexteLégalExactCode']].to_dict().values())[0].items())
# large_categs

## Update column in table

In [27]:
from postgres_connection import bulk_update_textes

In [28]:
# bulk_update_employees(textes_to_update = large_categs[0:1], )

bulk_update_textes(
    textes_to_update = large_categs, 
    column_name = table_column_name, 
    COUNTRY_NAME=COUNTRY_NAME)

Error updating Recueil Code général des impôts 2014 (Centre d_Etudes Juridiques Africaines): (psycopg2.errors.ForeignKeyViolation) insert or update on table "textes" violates foreign key constraint "textes_TexteLégalExactCode_fkey"
DETAIL:  Key (TexteLégalExactCode)=(nan) is not present in table "textes_legaux_exacts".

[SQL: UPDATE textes SET "TexteLégalExactCode"=%(TexteLégalExactCode)s WHERE textes."TexteCode" = %(textes_TexteCode)s]
[parameters: {'TexteLégalExactCode': 'nan', 'textes_TexteCode': 'Recueil Code général des impôts 2014 (Centre d_Etudes Juridiques Africaines)'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
Loi n°2013-021 (23.05.2013) Loi de finances rectificative 2013 (Droit-Afrique) column TexteLegalExactCode updated to Lég_Loi_Loi.
Loi n°2008-062 (03.12.2008) Loi de finances 2009 (LégiBurkina) column TexteLegalExactCode updated to Lég_Loi_Loi.
Loi n°2017-058 (20.12.2017) Code général des impôts 2017 (DGI) column TexteLegalExactCode updated to Lég_Loi_